In [5]:
import pandas as pd
import csv
import re
from bs4 import BeautifulSoup
import numpy as np

In [ ]:
df = pd.read_csv("src/archive/user_reviews.csv")

df = df.drop(["isVerified", "isSuperReviewer", "hasSpoilers", "hasProfanity", 
              "userRealm", 'reviewId', 'score', 'userDisplayName'], axis=1)

# df = df.groupby("userId").head(5)
#df.to_csv("src/processed/user_reviews.csv", index=False)

In [ ]:
# df = pd.read_csv("src/processed/user_reviews.csv")

In [ ]:
df_reduced = df.sample(frac=0.4, random_state=42)
# df_reduced.to_csv('df_reduced.csv', index=False)

In [ ]:
# userId e movieId indicizzati con valori numerici
def create_id_mapping(df, column):
    unique_ids = df[column].unique()
    id_mapping = {id_: idx for idx, id_ in enumerate(unique_ids)}
    return id_mapping

# valori sostituiti nei campi originali
user_id_mapping = create_id_mapping(df_reduced, 'userId')
df_reduced['userId'] = df_reduced['userId'].map(user_id_mapping)

movie_id_mapping = create_id_mapping(df_reduced, 'movieId')
df_reduced['movieId'] = df_reduced['movieId'].map(movie_id_mapping)


# Salvataggio mapping dati originali-indicizzati
# mapping per userId originale e indexed
user_mapping_df = pd.DataFrame(list(user_id_mapping.items()), columns=['userId_original', 'userId_index'])
user_mapping_df.to_csv('user_id_mapping.csv', index=False)

# mapping per movieId originale e indexed
movie_mapping_df = pd.DataFrame(list(movie_id_mapping.items()), columns=['movieId_original', 'movieId_index'])
movie_mapping_df.to_csv('movie_id_mapping.csv', index=False)

In [ ]:
# processing delle recensioni

# rimossi tag html
def remove_html_tags(text):
    if text:
        return BeautifulSoup(text, "html.parser").get_text()
    return text

# rimossi spazi bianchi multipli e a capo extra
def clean_spaces_and_line_breaks(text):
    if text:
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

# rimossi singoli e doppi apici extra
def remove_quotes(text):
    if text:
        text = text.strip('"')
        text = text.replace('"', '')
        return text
    return text

# rimossi caratteri speciali e emoji
def remove_non_ascii(text):
    if text:
        # ritorna solo caratteri ASCII
        return re.sub(r'[^\x00-\x7F]+', '', text)
    return text

def clean_csv_df(df):
    df_cleaned = df.copy()
    
    if 'quote' in df_cleaned.columns:
        df_cleaned['quote'] = df_cleaned['quote'].apply(remove_html_tags)
        df_cleaned['quote'] = df_cleaned['quote'].apply(clean_spaces_and_line_breaks)
        df_cleaned['quote'] = df_cleaned['quote'].apply(remove_quotes)
        df_cleaned['quote'] = df_cleaned['quote'].apply(remove_non_ascii)
    
    return df_cleaned


df_cleaned = clean_csv_df(df_reduced)
df_cleaned.to_csv('user_reviews_final_reduced.csv', index=False)

In [ ]:
# Read the CSV file
df = pd.read_csv("src/processed/user_reviews_processed.csv")
df = df.sample(10000, random_state=42)

# Check for NaN or infinite values in any column and remove those rows
df_cleaned = df.replace([np.inf, -np.inf], np.nan).dropna()

# Print the rows that were removed
invalid_rows = df[~df.index.isin(df_cleaned.index)]
print(invalid_rows)

df_cleaned['userId'] = df_cleaned['userId'].astype(int)
# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv('src/processed/user_reviews_processed_mini.csv', index=False)

C:\Users\User\AppData\Local\Temp\ipykernel_23172\265837714.py:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("src/processed/user_reviews_processed.csv")


Empty DataFrame
Columns: [movieId, rating, quote, creationDate, userId]
Index: []


In [ ]:
# # processing delle recensioni, applicata su csv 

# # rimossi tag html
# def remove_html_tags(text):
#     if text:
#         return BeautifulSoup(text, "html.parser").get_text()
#     return text

# # rimossi spazi bianchi multipli e a capo extra
# def clean_spaces_and_line_breaks(text):
#     if text:
#         text = re.sub(r'\s+', ' ', text).strip()
#         return text
#     return text

# # rimossi singoli e doppi apici extra
# def remove_quotes(text):
#     if text:
#         text = text.strip('"')
#         text = text.replace('"', '')
#         return text
#     return text

# # rimossi caratteri speciali e emoji
# def remove_non_ascii(text):
#     if text:
#         # ritorna solo caratteri ASCII
#         return re.sub(r'[^\x00-\x7F]+', '', text)
#     return text

# def clean_csv(input_file, output_file):
#     with open(input_file, newline='', encoding='utf-8') as infile:
#         reader = csv.reader(infile)
#         rows = []
#         current_row = None

#         for row in reader:
#             # se la riga è vuota, la ignoriamo
#             if not any(row):
#                 continue
            
#             # se la riga contiene solo una colonna, è una continuazione della precedente
#             if len(row) == 1 and current_row is not None:
#                 current_row[2] += " " + row[0] 
#             else:
#                 # aggiunta la riga precedente completata
#                 if current_row:
#                     current_row[2] = remove_html_tags(current_row[2])
#                     current_row[2] = clean_spaces_and_line_breaks(current_row[2]) 
#                     current_row[2] = remove_quotes(current_row[2])  
#                     current_row[2] = remove_non_ascii(current_row[2])
#                     rows.append(current_row)
                
#                 # nuova riga corrente
#                 current_row = row
        
#         # aggiunta l'ultima riga
#         if current_row:
#             current_row[2] = remove_html_tags(current_row[2])  
#             current_row[2] = clean_spaces_and_line_breaks(current_row[2]) 
#             current_row[2] = remove_quotes(current_row[2]) 
#             current_row[2] = remove_non_ascii(current_row[2])
#             rows.append(current_row)
    
#     with open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
#         writer = csv.writer(outfile)
#         writer.writerows(rows)

#     print(f"File pulito salvato come '{output_file}'.")


# df_reduced.to_csv('user_reviews_input.csv', index=False)
# input_file = 'user_reviews_input.csv' 
# output_file = 'user_reviews_cleaned.csv' 
# clean_csv(input_file, output_file)